In [ ]:
import tensorflow as tf

from train.train import Model
from modules.datasets import ImageTargetDataset
from modules.segm_transforms import convert_transforms

%load_ext autoreload
%autoreload 2

In [ ]:
device = 'GPU:0'
model_name = 'mobilenet_small'
n_class=1
load_model_path = '/workdir/data/experiments/mobilenetv3_tf_coco_pixart_supervisely_portrets_224_square_crop/model_best_0.005900000222027302.h5'
INPUT_SIZE = (224, 224)

mobilenet_model = Model(device=device,
                        model_name=model_name,
                        n_class=n_class,
                        input_shape=(1, INPUT_SIZE[0], INPUT_SIZE[1], 3),
                        old_model_path=load_model_path, shape=INPUT_SIZE)

### Basic tflite model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model.model)
tflite_model = converter.convert()
open("/workdir/data/tflite_models/converted_model_224_3.tflite",
     "wb").write(tflite_model)

### Full integer quantization of weights and activations

In [ ]:
val_trns = convert_transforms(scale_size=INPUT_SIZE)
val_dataset = ImageTargetDataset(['/workdir/data/datasets/portrets/val'], 1,
                                 shuffle=False,
                                 device='GPU:0',
                                 **val_trns,
                                 IMG_EXTN='.jpg',
                                 TRGT_EXTN='.png')

def representative_dataset_generator():
    for input_value, target_value in val_dataset:
            yield [input_value]


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenet_model.model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_generator
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()
open("/workdir/data/tflite_models/converted_model_224_3qu.tflite",
     "wb").write(tflite_quant_model)